In [1]:
import os
import sys
from os.path import join
from tqdm import tqdm

import pandas as pd
import numpy as np
import nfl_data_py as nfl

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(ROOT_DIR,'py'))

import util

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [2]:
# Define the path to the data folder
WEEK = 7
DATA_DIR = "../data/"
WEEKS = range(WEEK, WEEK+1)

df_game = pd.read_csv(join(DATA_DIR, "games.csv"))
df_play = pd.read_csv(join(DATA_DIR, "plays.csv"))
df_player_play = pd.read_csv(join(DATA_DIR, "player_play.csv"))
df_player = pd.read_csv(join(DATA_DIR, "players.csv"))

tracking_dfs = []
for wk in tqdm(WEEKS, desc="Loading tracking files"):
    df = pd.read_csv(join(DATA_DIR, f'tracking_week_{wk}.csv'))
    if 'week' not in df.columns:
        df.insert(3,'week',wk)
    tracking_dfs.append(df)
    
df_tracking = pd.concat(tracking_dfs, axis=0)

del tracking_dfs

util.uncamelcase_columns(df_game)
util.uncamelcase_columns(df_player)
util.uncamelcase_columns(df_play)
util.uncamelcase_columns(df_player_play)
util.uncamelcase_columns(df_tracking)

# standardize direction to be offense moving right
df_tracking, df_play = util.standardize_direction(df_tracking, df_play)

df_game = df_game.query('week==@WEEK').reset_index(drop=True)
game_ids = df_game['game_id'].unique().tolist()
df_player_play = df_player_play.query('game_id in @game_ids').reset_index(drop=True)

df_tracking = df_tracking.merge(df_player[['nfl_id','position']], on='nfl_id', how='left')

df_teams = nfl.import_team_desc()

Loading tracking files: 100%|██████████| 1/1 [00:05<00:00,  5.78s/it]


In [4]:
team_cols = ['team_abbr', 'team_color','team_color2','team_logo_wikipedia', 'team_wordmark']

if 'possession_team_color' not in df_play.columns:
    df_play = df_play.merge(
        right=df_teams[team_cols].rename(columns={
            'team_abbr':'possession_team',
            'team_color':'possession_team_color',
            'team_color2':'possession_team_color2',
            'team_logo_wikipedia':'possession_team_logo',
            'team_wordmark':'possession_team_wordmark'
        }),
        how='left',
        on='possession_team'
    )

if 'defensive_team_color' not in df_play.columns:
    df_play = df_play.merge(
        right=df_teams[team_cols].rename(columns={
            'team_abbr':'defensive_team',
            'team_color':'defensive_team_color',
            'team_color2':'defensive_team_color2',
            'team_logo_wikipedia':'defensive_team_logo',
            'team_wordmark':'defensive_team_wordmark',
        }),
        how='left',
        on='defensive_team'
    )

if 'home_team_abbr' not in df_play.columns:
    df_play = df_play.merge(
        right=df_game[['game_id','home_team_abbr','visitor_team_abbr']],
        how='left',
        on='game_id'
    ).rename(columns={
        'visitor_team_abbr':'away_team_abbr'
    })
    

if 'home_team_wordmark' not in df_play.columns:
    df_play['home_team_wordmark'] = np.where(
        df_play.home_team_abbr == df_play.possession_team, 
        df_play.possession_team_wordmark, 
        df_play.defensive_team_wordmark
    )

if 'home_team_logo' not in df_play.columns:
    df_play['home_team_logo'] = np.where(
        df_play.home_team_abbr == df_play.possession_team, 
        df_play.possession_team_logo, 
        df_play.defensive_team_logo
    )
    df_play['away_team_logo'] = np.where(
        df_play.home_team_abbr == df_play.possession_team, 
        df_play.defensive_team_logo,
        df_play.possession_team_logo
    )

if 'home_team_color' not in df_play.columns:
    df_play['home_team_color'] = np.where(
        df_play.home_team_abbr == df_play.possession_team, 
        df_play.possession_team_color, 
        df_play.defensive_team_color
    )
    df_play['away_team_color'] = np.where(
        df_play.home_team_abbr == df_play.possession_team, 
        df_play.defensive_team_color,
        df_play.possession_team_color
    )

if 'down_and_dist' not in df_play.columns:
    down_map = {
        1:'1st',
        2:'2nd',
        3:'3rd',
        4:'4th'
    }
    df_play['down_and_dist'] = df_play['down'].map(down_map) + ' & ' + df_play['yards_to_go'].astype(str)

if 'quarter_with_suffix' not in df_play.columns:
    quarter_map = {
        1:'1st',
        2:'2nd',
        3:'3rd',
        4:'4th',
    }
    df_play['quarter_with_suffix'] = df_play['quarter'].map(quarter_map)


In [53]:
df_teams.query('team_abbr=="DAL"').values

array([['DAL', 'Dallas Cowboys', 1200, 'Cowboys', 'NFC', 'NFC East',
        '#002244', '#B0B7BC', '#acc0c6', '#a5acaf',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Dallas_Cowboys.svg/100px-Dallas_Cowboys.svg.png',
        'https://a.espncdn.com/i/teamlogos/nfl/500/dal.png',
        'https://github.com/nflverse/nflverse-pbp/raw/master/wordmarks/DAL.png',
        'https://github.com/nflverse/nflverse-pbp/raw/master/NFC.png',
        'https://raw.githubusercontent.com/nflverse/nflverse-pbp/master/NFL.png',
        'https://github.com/nflverse/nflverse-pbp/raw/master/squared_logos/DAL.png']],
      dtype=object)

In [137]:
df_play.head()

,game_id,play_id,play_description,quarter,down,yards_to_go,possession_team,defensive_team,yardline_side,yardline_number,game_clock,pre_snap_home_score,pre_snap_visitor_score,play_nullified_by_penalty,absolute_yardline_number,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,expected_points,offense_formation,receiver_alignment,play_clock_at_snap,pass_result,pass_length,target_x,target_y,play_action,dropback_type,dropback_distance,pass_location_type,time_to_throw,time_in_tackle_box,time_to_sack,pass_tipped_at_line,unblocked_pressure,qb_spike,qb_kneel,qb_sneak,rush_location_type,penalty_yards,pre_penalty_yards_gained,yards_gained,home_team_win_probability_added,visitor_team_win_probility_added,expected_points_added,is_dropback,pff_run_concept_primary,pff_run_concept_secondary,pff_run_pass_option,pff_pass_coverage,pff_man_zone,play_direction,possession_team_color,possession_team_logo,defensive_team_color,defensive_team_logo,down_and_dist,quarter_with_suffix,home_team_abbr,away_team_abbr
0,2022102302,2655,(1:54) (Shotgun) J.Burrow pass short middle to...,3,1,10,CIN,ATL,CIN,21,01:54,35,17,N,31,0.982017,0.017983,0.719313,EMPTY,3x2,10.0,C,6.0,36.69,16.51,False,TRADITIONAL,2.40,INSIDE_BOX,2.99,2.99,NaN,False,False,False,0,NaN,NaN,NaN,9,9,0.004634,-0.004634,0.702717,True,NaN,NaN,0,Cover-3,Zone,right,#FB4F14,https://upload.wikimedia.org/wikipedia/commons...,#A71930,https://upload.wikimedia.org/wikipedia/en/thum...,1st & 10,3rd,CIN,ATL
1,2022102306,3240,(12:55) (Shotgun) A.Dillon left tackle to WAS ...,4,2,10,GB,WAS,WAS,46,12:55,20,14,N,64,0.630746,0.369254,2.416086,SHOTGUN,3x1,4.0,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,INSIDE_LEFT,NaN,0,0,0.053308,-0.053308,-0.962012,False,PULL LEAD,NaN,0,Cover-6 Right,Zone,right,#203731,https://upload.wikimedia.org/wikipedia/commons...,#5A1414,https://upload.wikimedia.org/wikipedia/commons...,2nd & 10,4th,WAS,GB
2,2022102308,3304,"(2:57) (No Huddle, Shotgun) D.Mills pass incom...",4,2,1,HOU,LV,HOU,29,02:57,38,20,N,39,0.998002,0.001998,1.675212,EMPTY,3x2,27.0,I,11.0,42.66,48.29,False,TRADITIONAL,2.16,INSIDE_BOX,2.77,2.77,NaN,False,False,False,0,NaN,NaN,NaN,0,0,0.000007,-0.000007,-0.373242,True,NaN,NaN,0,Quarters,Zone,right,#03202F,https://upload.wikimedia.org/wikipedia/en/thum...,#000000,https://upload.wikimedia.org/wikipedia/en/thum...,2nd & 1,4th,LV,HOU
3,2022102302,1437,"(4:49) T.Allgeier left guard for 1 yard, TOUCH...",2,3,1,ATL,CIN,CIN,1,04:49,21,0,N,109,0.938741,0.061259,5.729890,I_FORM,2x1,15.0,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,OUTSIDE_LEFT,NaN,1,1,0.008083,-0.008083,1.270110,False,MAN,LEAD,0,Goal Line,Other,left,#A71930,https://upload.wikimedia.org/wikipedia/en/thum...,#FB4F14,https://upload.wikimedia.org/wikipedia/commons...,3rd & 1,2nd,CIN,ATL
4,2022102000,1707,"(1:30) (No Huddle, Shotgun) A.Dalton scrambles...",2,2,2,NO,ARI,NO,33,01:30,20,14,N,43,0.716728,0.283272,0.707889,SHOTGUN,2x2,30.0,R,NaN,NaN,NaN,False,SCRAMBLE,3.10,NaN,NaN,NaN,NaN,NaN,False,NaN,0,False,INSIDE_RIGHT,NaN,14,14,-0.022215,0.022215,0.434729,True,NaN,NaN,0,2-Man,Man,right,#D3BC8D,https://upload.wikimedia.org/wikipedia/commons...,#97233F,https://upload.wikimedia.org/wikipedia/en/thum...,2nd & 2,2nd,ARI,NO


In [7]:
df_tracking.head()

,game_id,play_id,nfl_id,week,display_name,frame_id,frame_type,time,jersey_number,club,play_direction,x,y,s,a,dis,o,dir,event
0,2022102400,66,37087.0,7,Robert Quinn,1,BEFORE_SNAP,2022-10-25 00:16:40.6,94.0,CHI,right,27.93,28.63,1.29,0.98,0.12,200.77,179.85,huddle_break_offense
1,2022102400,66,37087.0,7,Robert Quinn,2,BEFORE_SNAP,2022-10-25 00:16:40.7,94.0,CHI,right,27.80,28.63,1.38,0.76,0.13,197.13,179.67,NaN
2,2022102400,66,37087.0,7,Robert Quinn,3,BEFORE_SNAP,2022-10-25 00:16:40.8,94.0,CHI,right,27.66,28.63,1.45,0.57,0.14,195.71,178.97,NaN
3,2022102400,66,37087.0,7,Robert Quinn,4,BEFORE_SNAP,2022-10-25 00:16:40.9,94.0,CHI,right,27.52,28.63,1.46,0.27,0.14,195.71,179.12,NaN
4,2022102400,66,37087.0,7,Robert Quinn,5,BEFORE_SNAP,2022-10-25 00:16:41,94.0,CHI,right,27.37,28.64,1.46,0.13,0.14,197.27,178.39,NaN


In [8]:
df_game.head()

,game_id,season,week,game_date,game_time_eastern,home_team_abbr,visitor_team_abbr,home_final_score,visitor_final_score
0,2022102000,2022,7,10/20/2022,20:15:00,ARI,NO,42,34
1,2022102300,2022,7,10/23/2022,13:00:00,BAL,CLE,23,20
2,2022102301,2022,7,10/23/2022,13:00:00,CAR,TB,21,3
3,2022102302,2022,7,10/23/2022,13:00:00,CIN,ATL,35,17
4,2022102303,2022,7,10/23/2022,13:00:00,DAL,DET,24,6


In [9]:
df_player.head()

,nfl_id,height,weight,birth_date,college_name,position,display_name
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan


In [10]:
df_play.head()

,game_id,play_id,play_description,quarter,down,yards_to_go,possession_team,defensive_team,yardline_side,yardline_number,game_clock,pre_snap_home_score,pre_snap_visitor_score,play_nullified_by_penalty,absolute_yardline_number,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,expected_points,offense_formation,receiver_alignment,play_clock_at_snap,pass_result,pass_length,target_x,target_y,play_action,dropback_type,dropback_distance,pass_location_type,time_to_throw,time_in_tackle_box,time_to_sack,pass_tipped_at_line,unblocked_pressure,qb_spike,qb_kneel,qb_sneak,rush_location_type,penalty_yards,pre_penalty_yards_gained,yards_gained,home_team_win_probability_added,visitor_team_win_probility_added,expected_points_added,is_dropback,pff_run_concept_primary,pff_run_concept_secondary,pff_run_pass_option,pff_pass_coverage,pff_man_zone,play_direction
0,2022102302,2655,(1:54) (Shotgun) J.Burrow pass short middle to...,3,1,10,CIN,ATL,CIN,21,01:54,35,17,N,31,0.982017,0.017983,0.719313,EMPTY,3x2,10.0,C,6.0,36.69,16.51,False,TRADITIONAL,2.40,INSIDE_BOX,2.99,2.99,NaN,False,False,False,0,NaN,NaN,NaN,9,9,0.004634,-0.004634,0.702717,True,NaN,NaN,0,Cover-3,Zone,right
1,2022102306,3240,(12:55) (Shotgun) A.Dillon left tackle to WAS ...,4,2,10,GB,WAS,WAS,46,12:55,20,14,N,64,0.630746,0.369254,2.416086,SHOTGUN,3x1,4.0,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,INSIDE_LEFT,NaN,0,0,0.053308,-0.053308,-0.962012,False,PULL LEAD,NaN,0,Cover-6 Right,Zone,right
2,2022102308,3304,"(2:57) (No Huddle, Shotgun) D.Mills pass incom...",4,2,1,HOU,LV,HOU,29,02:57,38,20,N,39,0.998002,0.001998,1.675212,EMPTY,3x2,27.0,I,11.0,42.66,48.29,False,TRADITIONAL,2.16,INSIDE_BOX,2.77,2.77,NaN,False,False,False,0,NaN,NaN,NaN,0,0,0.000007,-0.000007,-0.373242,True,NaN,NaN,0,Quarters,Zone,right
3,2022102302,1437,"(4:49) T.Allgeier left guard for 1 yard, TOUCH...",2,3,1,ATL,CIN,CIN,1,04:49,21,0,N,109,0.938741,0.061259,5.729890,I_FORM,2x1,15.0,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,OUTSIDE_LEFT,NaN,1,1,0.008083,-0.008083,1.270110,False,MAN,LEAD,0,Goal Line,Other,left
4,2022102000,1707,"(1:30) (No Huddle, Shotgun) A.Dalton scrambles...",2,2,2,NO,ARI,NO,33,01:30,20,14,N,43,0.716728,0.283272,0.707889,SHOTGUN,2x2,30.0,R,NaN,NaN,NaN,False,SCRAMBLE,3.10,NaN,NaN,NaN,NaN,NaN,False,NaN,0,False,INSIDE_RIGHT,NaN,14,14,-0.022215,0.022215,0.434729,True,NaN,NaN,0,2-Man,Man,right


In [11]:
[c for c in df_player_play.columns if 'motion' in c]

['in_motion_at_ball_snap', 'motion_since_lineset']

In [12]:
[c for c in df_player_play.columns if 'shift' in c]

['shift_since_lineset']

In [13]:
df_player_play.head()

,game_id,play_id,nfl_id,team_abbr,had_rush_attempt,rushing_yards,had_dropback,passing_yards,sack_yards_as_offense,had_pass_reception,receiving_yards,was_targetted_receiver,yardage_gained_after_the_catch,fumbles,fumble_lost,fumble_out_of_bounds,assisted_tackle,forced_fumble_as_defense,half_sack_yards_as_defense,pass_defensed,quarterback_hit,sack_yards_as_defense,safety_as_defense,solo_tackle,tackle_assist,tackle_for_a_loss,tackle_for_a_loss_yardage,had_interception,interception_yards,fumble_recoveries,fumble_recovery_yards,penalty_yards,penalty_names,was_initial_pass_rusher,caused_pressure,time_to_pressure_as_pass_rusher,get_off_time_as_pass_rusher,in_motion_at_ball_snap,shift_since_lineset,motion_since_lineset,was_running_route,route_ran,blocked_player_n_f_l_id1,blocked_player_n_f_l_id2,blocked_player_n_f_l_id3,pressure_allowed_as_blocker,time_to_pressure_allowed_as_blocker,pff_defensive_coverage_assignment,pff_primary_defensive_coverage_matchup_nfl_id,pff_secondary_defensive_coverage_matchup_nfl_id
0,2022102000,56,37084,ARI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1.0,False,NaN,1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022102000,56,44848,ARI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CFL,54476.0,NaN
2,2022102000,56,46521,ARI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOL,NaN,NaN
3,2022102000,56,47816,ARI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3L,NaN,NaN
4,2022102000,56,47848,ARI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1.0,False,NaN,0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
track_cols = ['game_id', 'play_id', 'event']
df_tracking['motion_play'] = df_tracking.groupby(['game_id', 'play_id'])['event'].transform(lambda x: 1 if 'man_in_motion' in x.values else 0)
df_tracking['shift_play'] = df_tracking.groupby(['game_id', 'play_id'])['event'].transform(lambda x: 1 if 'shift' in x.values else 0)
df_motion_shift = df_tracking.drop_duplicates(subset=['game_id', 'play_id'])[['game_id', 'play_id', 'motion_play','shift_play']].reset_index(drop=True)

df_player_play['motion'] = np.where(df_player_play.in_motion_at_ball_snap | df_player_play.motion_since_lineset, 1, 0)
df_player_play['motion_player'] = df_player_play.groupby(['game_id', 'play_id'])['motion'].transform(lambda x: 1 if x.sum() > 0 else 0)
df_player_play['shift_player'] = df_player_play.groupby(['game_id', 'play_id'])['shift_since_lineset'].transform(lambda x: 1 if x.sum() > 0 else 0)
player_play_cols = ['game_id', 'play_id', 'motion_player', 'shift_player']
df_motion_shift = df_motion_shift.merge(df_player_play[player_play_cols].drop_duplicates(['game_id', 'play_id']), on=['game_id', 'play_id'], how='left')

In [15]:
df_motion_shift.motion_play.value_counts()

0    1078
1     587
Name: motion_play, dtype: int64

In [16]:
df_motion_shift.motion_player.value_counts()

0    990
1    675
Name: motion_player, dtype: int64

In [17]:
df_motion_shift.shift_play.value_counts()

0    1366
1     299
Name: shift_play, dtype: int64

In [18]:
df_motion_shift.shift_player.value_counts()

0    1304
1     361
Name: shift_player, dtype: int64

In [25]:
df_tracking[['game_id','play_id']].value_counts()

game_id     play_id
2022102306  1432       11684
            4181        9384
2022102302  3081        9338
2022102310  359         8211
            4097        8188
2022102304  1197        7268
2022102305  3031        7061
2022102301  3273        6946
2022102302  2606        6946
            2896        6785
2022102400  3027        6601
2022102302  2582        6555
2022102301  818         6394
2022102304  725         6325
2022102302  3337        6325
2022102301  1380        6302
2022102308  2464        6256
2022102302  970         6210
2022102300  765         6210
2022102304  2125        6141
2022102000  2556        6118
2022102308  671         6095
2022102300  1338        6072
2022102308  185         6072
2022102301  2256        6026
2022102311  3141        6026
            3275        6003
2022102308  2116        5980
2022102400  238         5934
2022102304  80          5911
2022102303  2826        5911
2022102302  2285        5842
2022102308  137         5819
2022102400  609        

In [5]:
import colorsys

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle, Polygon
from matplotlib.font_manager import FontProperties
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.patches import Wedge
from matplotlib.colors import to_rgba
from IPython.display import HTML
import urllib
import PIL

def hex_to_rgb(hex_color):
    # Strip the '#' if it exists
    hex_color = hex_color.lstrip('#')
    
    # Convert hex to RGB
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def luminance(rgb):
    r, g, b = [x / 255.0 for x in rgb]  # Normalize to [0, 1]
    r = (r / 12.92) if r <= 0.03928 else ((r + 0.055) / 1.055) ** 2.4
    g = (g / 12.92) if g <= 0.03928 else ((g + 0.055) / 1.055) ** 2.4
    b = (b / 12.92) if b <= 0.03928 else ((b + 0.055) / 1.055) ** 2.4
    return 0.2126 * r + 0.7152 * g + 0.0722 * b

# Function to calculate contrast ratio between two hex colors
def contrast_ratio(hex_color1, hex_color2):
    # Convert hex to RGB (normalized to [0, 1])
    rgb1 = hex_to_rgb(hex_color1)
    rgb2 = hex_to_rgb(hex_color2)
    
    # Calculate luminance of each color
    lum1 = luminance(rgb1)
    lum2 = luminance(rgb2)
    
    # Calculate contrast ratio (lighter / darker)
    lighter = max(lum1, lum2)
    darker = min(lum1, lum2)
    
    return (lighter + 0.05) / (darker + 0.05)

class NFLPlayAnimation:
    def __init__(self, tracking_data, play_data, show_scoreboard=True, clock_rolling=True, player_display_type='dots', show_player_legend=False):
        self.tracking_data = tracking_data
        self.play_data = play_data
        self.show_scoreboard = show_scoreboard
        self.clock_rolling = clock_rolling
        self.player_display_type = player_display_type # options: 'dots', 'positions', 'jerseys
        self.show_player_legend = show_player_legend
        if self.show_player_legend:
            self.fig, self.ax = plt.subplots(figsize=(14, 8))
        else:
            self.fig, self.ax = plt.subplots(figsize=(12, 8))
        self.aspect_ratio = 1
        self.y_delta = 35
        self.x_limit_min = 0
        self.x_limit_max = 53.3
        self.numbers_font = FontProperties(fname='../data/fonts/clarendon_bold.otf')
        self.scoreboard_height = 3
        img = PIL.Image.open(urllib.request.urlopen(play_data['home_team_logo']))
        w,h = img.size
        w_new = min(int(w * (70/h)), 75)
        self.home_img = OffsetImage(img.resize((w_new,70)).crop((0,10,w_new,70)))
        img = PIL.Image.open(urllib.request.urlopen(play_data['away_team_logo']))
        w,h = img.size
        w_new = min(int(w * (70/h)), 75)
        self.away_img = OffsetImage(img.resize((w_new,70)).crop((0,10,w_new,70)))
        self.home_wordmark = OffsetImage(PIL.Image.open(urllib.request.urlopen(play_data['home_team_wordmark'])), zoom=1)
        self.home_wordmark_rotated= OffsetImage(PIL.Image.open(urllib.request.urlopen(play_data['home_team_wordmark'])).rotate(180), zoom=1)
        self.poss_tm_color = play_data['possession_team_color']
        self.poss_tm_edge_color = play_data['possession_team_color2']
        cr_1 = contrast_ratio(play_data['possession_team_color'], play_data['defensive_team_color'])
        cr_2 = contrast_ratio(play_data['possession_team_color'], play_data['defensive_team_color2'])
        self.def_tm_color = play_data['defensive_team_color'] if cr_1 > cr_2  else play_data['defensive_team_color2']
        self.def_tm_edge_color = play_data['defensive_team_color2'] if cr_1 > cr_2  else play_data['defensive_team_color']

        if self.show_scoreboard:
            self.play_clocks = dict()
            snap_frame_id = self.tracking_data.query('event=="ball_snap"')['frame_id'].values[0]
            for fid in self.tracking_data.frame_id.unique():
                if fid <= snap_frame_id:
                    self.play_clocks[fid] = int(self.play_data['play_clock_at_snap'] + (snap_frame_id - fid) / 10 - .1)
                else:
                    self.play_clocks[fid] = 40

            # create game clock (in seconds) using self.play_data['game_clock']
            self.game_clocks = dict()
            game_clock_min = int(self.play_data['game_clock'].split(':')[0])
            game_clock_sec = int(self.play_data['game_clock'].split(':')[1])
            if clock_rolling:
                game_clock_sec = game_clock_sec + ((snap_frame_id % 10) / 10) + (snap_frame_id // 10)
                if game_clock_sec >= 60:
                    game_clock_min += game_clock_sec // 60
                    game_clock_sec = game_clock_sec % 60
            for fid in self.tracking_data.frame_id.unique():
                if fid < snap_frame_id:
                    if self.clock_rolling:
                        if game_clock_sec < 0:
                            if game_clock_min == 0:
                                game_clock_sec = 0
                                game_clock_min = 0
                            else:
                                game_clock_min -= 1
                                game_clock_sec = 59.9
                        else:
                            game_clock_sec -= 0.1
                        self.game_clocks[fid] = f'{int(game_clock_min):02}:{int(game_clock_sec):02}'
                    else:
                        self.game_clocks[fid] = self.play_data['game_clock']
                else:
                    if game_clock_sec < 0:
                        if game_clock_min == 0:
                            game_clock_sec = 0
                            game_clock_min = 0
                        else:
                            game_clock_min -= 1
                            game_clock_sec = 59.9
                    else:
                        game_clock_sec -= 0.1
                    self.game_clocks[fid] = f'{int(game_clock_min):02}:{int(game_clock_sec):02}'

    def plot_field(self):
        """Plot the NFL field layout with square aspect ratio."""
        # Hard limits for the x-axis (do not exceed the field width)
        if self.show_player_legend:
            self.ax.set_xlim(self.x_limit_min, self.x_limit_max + 15)
        else:
            self.ax.set_xlim(self.x_limit_min, self.x_limit_max)
        # Hard limits for the y-axis, updated dynamically later
        self.ax.set_ylim(self.y_limit_min, self.y_limit_min + self.y_delta)
        # self.ax.set_aspect(self.aspect_ratio)

        # Set y-axis ticks every 5 yards, excluding end zones
        yticks = [i for i in range(0, 121, 5) if i not in [5, 115]]
        self.ax.set_yticks(yticks)
        
        # Remove x-axis ticks
        self.ax.set_xticks([self.x_limit_min, self.x_limit_max])

        self.ax.grid(True, which='major', axis='y', color='white', linewidth=2)
        self.ax.grid(True, which='major', axis='x', color='white', linewidth=2)
        
        # Set background to light gray
        self.ax.set_facecolor('lightgray')
        
        # Remove plot spines (borders)
        for spine in self.ax.spines.values():
            spine.set_visible(False)

        # Set tick parameters and hide tick labels
        self.ax.tick_params(left=False, right=False, top=False, bottom=False, labelleft=False, labelbottom=False)

        # Draw vertical white lines on the edges of the field
        # self.ax.axvline(x=self.x_limit_min, color='white', linewidth=2)
        # self.ax.axvline(x=self.x_limit_max, color='white', linewidth=2)

        # Draw horizontal lines at yard lines 0 and 120 (end zones)
        # self.ax.axhline(y=0, color='white', linewidth=2)
        # self.ax.axhline(y=120, color='white', linewidth=2)

        # Draw line of scimmage
        self.ax.axhline(y=self.play_data['absolute_yardline_number'], color='blue', linewidth=2)

        # Draw first down line
        self.ax.axhline(y=self.play_data['absolute_yardline_number'] + self.play_data['yards_to_go'], color='yellow', linewidth=2)

        # Add yard markers
        for y in range(11, 110, 1):
            if y % 5 != 0:
                centerfield = self.x_limit_max / 2
                left_outer = Rectangle((1/2, y - 0.05), 2/3, 0.04, color='white')
                left_inner = Rectangle((centerfield - (37/12 + 1/3), y - 0.05), 2/3, 0.04, color='white')
                right_inner = Rectangle((centerfield + (37/12 - 1/3), y - 0.05), 2/3, 0.04, color='white')
                right_outer = Rectangle((self.x_limit_max - 7/6, y - 0.05), 2/3, 0.04, color='white')

            for hash_mark in [left_outer, left_inner, right_inner, right_outer]:
                self.ax.add_patch(hash_mark)

        # Add yardline numbers
        yardline_labels = {20: "1 0", 30: "2 0", 40: "3 0", 50: "4 0", 60: "5 0", 70: "4 0", 80: "3 0", 90: "2 0", 100: "1 0"}
        for y, label in yardline_labels.items():
            # Add yardline numbers on the left side
            self.ax.text(
                12, y, 
                label, 
                ha='center', va='center', 
                fontsize=30, 
                color='white', 
                rotation=-90, 
                fontproperties=self.numbers_font
            )
            # Add yardline numbers on the right side
            self.ax.text(
                self.x_limit_max - 12, y,
                label, 
                ha='center', va='center', 
                fontsize=30, 
                color='white', 
                rotation=90, 
                fontproperties=self.numbers_font
            )

            if y > 60:
                # plot arrows gonig up
                left_triangle = Polygon([[12, y + 1.8], [12.2, y + 2.55], [12.4, y + 1.8]], color='white')
                right_triangle = Polygon([[self.x_limit_max - 12, y + 1.8], [self.x_limit_max - 12.2, y + 2.55], [self.x_limit_max - 12.4, y + 1.8]], color='white')
                self.ax.add_patch(left_triangle)
                self.ax.add_patch(right_triangle)
            elif y < 60:
                # plot arrows going down
                left_triangle = Polygon([[12, y - 1.8], [12.2, y - 2.55], [12.4, y - 1.8]], color='white')
                right_triangle = Polygon([[self.x_limit_max - 12, y - 1.8], [self.x_limit_max - 12.2, y - 2.55], [self.x_limit_max - 12.4, y - 1.8]], color='white')
                self.ax.add_patch(left_triangle)
                self.ax.add_patch(right_triangle)

        # if y_limit_min + y_delta > 110, plot home_team_wordmark image from url in endzone
        if self.y_limit_min + self.y_delta > 110:
            # plot darker endzone
            endzone = Rectangle((0, 110), self.x_limit_max, 10, color='grey')
            self.ax.add_patch(endzone)
            self.add_logo(self.x_limit_max / 2, 115, self.home_wordmark, ord=7)

        if self.y_limit_min < 10:
            # plot darker endzone
            endzone = Rectangle((0, 0), self.x_limit_max, 10, color='grey')
            self.ax.add_patch(endzone)
            self.add_logo(self.x_limit_max / 2, 5, self.home_wordmark_rotated, ord=7)
        
    def plot_scoreboard(self, frame_id=None):
        x_interval = self.x_limit_max / 4
            
        rect_team1 = Rectangle(
            (0, self.y_limit_min),
            x_interval,
            self.scoreboard_height,
            color=self.play_data['away_team_color'],
            zorder=6
        )
        rect_team2 = Rectangle(
            (x_interval, self.y_limit_min),
            x_interval * 2,
            self.scoreboard_height,    
            color=self.play_data['home_team_color'],
            zorder=6 
        )
        rect_time = Rectangle(
            (x_interval * 2, self.y_limit_min),
            x_interval * 3 - 4,
            self.scoreboard_height,
            color='#1a1817',
            zorder=6 
        )
        # if clock uner 5 seconds, change color to red
        pc_color = 'grey'
        if frame_id==None:
            frame_id = self.tracking_data.frame_id.min()
        if self.play_clocks[frame_id] <= 5:
            pc_color = 'red'
        rect_play_clock = Rectangle(
            (x_interval * 3 - 4, self.y_limit_min),
            x_interval * 3,
            self.scoreboard_height,
            color=pc_color,
            zorder=6 
        )
        rect_down_dist = Rectangle(
            (x_interval * 3, self.y_limit_min),
            self.x_limit_max,
            self.scoreboard_height,
            color=self.play_data['possession_team_color'],
            zorder=6 
        )
        
        for rect in [rect_team1, rect_team2, rect_time, rect_play_clock, rect_down_dist]:
            self.ax.add_patch(rect)

        txt_height = self.y_limit_min + self.scoreboard_height / 2
        # Plot the home and away team scores
        self.ax.text(
            x_interval / 2 + 2.5, txt_height,
            f'{self.play_data["away_team_abbr"]}    {self.play_data["pre_snap_visitor_score"]}',
            ha='center', va='center', 
            fontsize=20, 
            fontweight='bold',
            color='white',
            zorder=8,
        )

        self.ax.text(
            x_interval * 1.5 + 2.5, txt_height,
            f'{self.play_data["home_team_abbr"]}    {self.play_data["pre_snap_home_score"]}',
            ha='center', va='center', 
            fontsize=20, 
            fontweight='bold',
            color='white',
            zorder=7,
        )

        # Add time to r3
        self.ax.text(
            x_interval * 2 + (x_interval / 2 - 2), txt_height,
            f'{self.play_data["quarter_with_suffix"]}  {self.game_clocks[frame_id]}',
            ha='center', va='center', 
            fontsize=20, 
            fontweight='bold',
            color='white',
            zorder=7,
        )

        # add text to r4 (play clock)
        self.ax.text(
            x_interval * 3 - 2, txt_height,
            f'{self.play_clocks[frame_id]:02}',
            ha='center', va='center', 
            fontsize=20, 
            fontweight='bold',
            color='white',
            zorder=7,
        )
        
        # Add down and distance to r5
        self.ax.text(
            x_interval * 3.5, txt_height,
            self.play_data['down_and_dist'],
            ha='center', va='center', 
            fontsize=20, 
            fontweight='bold',
            color='white',
            zorder=7,
        )

        # Add team logos next to the scores
        self.add_logo(3, txt_height - .5, self.away_img, ord=7)
        self.add_logo(x_interval + 3, txt_height - .5, self.home_img, ord=7)

    def plot_player_legend(self):
        y = self.y_limit_min + self.y_delta
        if self.show_scoreboard: y -= self.scoreboard_height
        rect_heading = Rectangle(
            (self.x_limit_max, y),
            15,
            .25,
            color='black',
            zorder=7
        )
        self.ax.add_patch(rect_heading)

        h = self.y_delta if self.show_scoreboard else self.y_delta + self.scoreboard_height
        rect_body = Rectangle(
            (self.x_limit_max, self.y_limit_min),
            15,
            h,
            color='white',
            zorder=6
        )
        self.ax.add_patch(rect_body)

        self.ax.text(
            self.x_limit_max + 7.5, y + 1.75,
            'Player Legend',
            ha='center', va='center',
            fontsize=25,
            fontweight='bold',
            color='black',
            zorder=8,
            font
        )

    def add_logo(self, x, y, imagebox, ord=8):
        """Helper function to add team logo to the plot."""
        # Create an OffsetImage with the logo and set a proper zoom to fit it in the scoreboard
        ab = AnnotationBbox(imagebox, (x, y), frameon=False, zorder=ord)
        self.ax.add_artist(ab)
        
    def init_animation(self):
        """Initialize the animation (empty field)."""
        self.plot_field()
        if self.show_scoreboard: self.plot_scoreboard()
        if self.show_player_legend: self.plot_player_legend()
        return self.ax
    
    def update_frame(self, frame_id):
        """Update the plot for each frame."""
        self.ax.clear()

        # Plot the field
        self.plot_field()

        # Get data for the current frame
        frame_data = self.tracking_data[self.tracking_data['frame_id'] == frame_id]
        
        ball_y = None
        
        # Plot players and football
        for club, group in frame_data.groupby('club'):
            if club == 'football':
                size = 140
                if self.player_display_type == 'dots':
                    size = 80
                ball_y = group['y'].iloc[0]
                # Plot football as a regular circle
                self.ax.scatter(group['x'], group['y'], color='brown', marker='d', s=size, edgecolors='black', zorder=6)
                self.ax.scatter(group['x'], group['y'], color='white', marker='|', s=size / 3, zorder=6)
            else:
                # Assign teams different colors
                if club == self.play_data['possession_team']:
                    color = to_rgba(self.poss_tm_color)
                    ec = to_rgba(self.poss_tm_edge_color)
                else:
                    color = to_rgba(self.def_tm_color)
                    ec = to_rgba(self.def_tm_edge_color)

                if self.player_display_type == 'dots':
                    # Plot players as dots
                    self.ax.scatter(group['x'], group['y'], color=color, s=80, edgecolors=ec, zorder=6)
                elif self.player_display_type in ['positions', 'jerseys']:
                    # Plot players as circles with a flat side and a front "half-square"
                    for _, player in group.iterrows():
                        orientation = player['o']  # Convert radians to degrees
                        radius = 0.7  # Radius for the players
                        x, y = player['x'], player['y']
                                                
                        # Create the Wedge for each player (flat side 180 degrees opposite orientation)
                        wedge = Wedge((x, y), radius, theta1=orientation-90, theta2=orientation+90, color=color, zorder=5, ec=ec)

                        # Add the wedge to the axis
                        self.ax.add_patch(wedge)

                        # Calculate the half-square vertices
                        square_length = radius  # Length of the half-square extension in front of the flat side of the circle
                        angle_rad = np.radians(orientation)

                        # Calculate the direction vector for the front of the player (where the square will extend)
                        dx = square_length * np.cos(angle_rad)
                        dy = square_length * np.sin(angle_rad)

                        # Compute points along the flat edge of the circle (aligned with the player's orientation)
                        left_edge_x = x + radius * np.cos(angle_rad - np.pi/2)  # Left side of the flat edge
                        left_edge_y = y + radius * np.sin(angle_rad - np.pi/2)
                        right_edge_x = x + radius * np.cos(angle_rad + np.pi/2)  # Right side of the flat edge
                        right_edge_y = y + radius * np.sin(angle_rad + np.pi/2)

                        # Define the four corners of the square, extending from the flat side
                        # These corners are along the flat edge and then extend forward in the direction of the player's orientation
                        corners = [
                            (right_edge_x, right_edge_y),  # Right side of flat edge
                            (left_edge_x, left_edge_y),    # Left side of flat edge
                            (left_edge_x - dx, left_edge_y - dy),  # Front-left (extend forward)
                            (right_edge_x - dx, right_edge_y - dy)  # Front-right (extend forward)
                        ]

                        # Create the half-square polygon and add it to the axis
                        half_square = Polygon(corners, closed=True, color=color, zorder=5, ec=ec)
                        self.ax.add_patch(half_square)

                        # add rectangular patch where circle and square meet
                        radius -= 0.08
                        left_edge_x = x + radius * np.cos(angle_rad - np.pi/2)  # Left side of the flat edge
                        left_edge_y = y + radius * np.sin(angle_rad - np.pi/2)
                        right_edge_x = x + radius * np.cos(angle_rad + np.pi/2)  # Right side of the flat edge
                        right_edge_y = y + radius * np.sin(angle_rad + np.pi/2)
                        corners = [
                            (right_edge_x + .1 * dx, right_edge_y + .1 * dy),
                            (left_edge_x + .1 * dx, left_edge_y + .1 * dy),
                            (left_edge_x -.1 * dx, left_edge_y - .1 * dy),
                            (right_edge_x - .1 * dx, right_edge_y -.1 * dy)
                        ]
                        rect = Polygon(corners, closed=True, color=color, zorder=5)
                        self.ax.add_patch(rect)

                        # Plot the player's jersey number, centered at (x, y)
                        if self.player_display_type == 'jerseys':
                            jersey_number = int(player['jersey_number'])  # Convert float to int
                            self.ax.text(x, y, str(jersey_number), color='white', ha='center', va='center', fontweight='bold', fontsize=12, zorder=6)
                        else:
                            position = player['position']  # Get the player's position
                            self.ax.text(x, y, position, color='white', ha='center', va='center', fontweight='bold', fontsize=9, zorder=6)
                else:
                    raise ValueError("Invalid player_display_type. Must be one of 'dots', 'positions', or 'jersey_numbers'.")

            # Dynamically adjust the y-axis limit based on the ball's y position
            if ball_y is not None:
                if ball_y < self.y_limit_min + 10:  # Ball near the bottom
                    self.y_limit_min = max(0, ball_y - 10)
                elif ball_y > self.y_limit_min + self.y_delta - 10:  # Ball near the top
                    self.y_limit_min = min(120 - self.y_delta, ball_y - self.y_delta + 10)
                self.ax.set_ylim(self.y_limit_min, self.y_limit_min + self.y_delta)

            if self.show_scoreboard: 
                self.plot_scoreboard(frame_id)

            if self.show_player_legend:
                self.plot_player_legend()

        return self.ax

    def animate_play(self):
        """Create the animation of the play."""
        # Set initial y-axis limit to follow the football at the snap
        self.y_limit_min = round(self.tracking_data[
            (self.tracking_data['club'] == 'football') & (self.tracking_data['event'] == 'ball_snap')
        ]['y'].iloc[0] - 10, 2)

        if self.show_scoreboard:
            self.y_limit_min -= self.scoreboard_height

        frame_ids = self.tracking_data['frame_id'].unique()
        
        # Generate the animation
        ani = animation.FuncAnimation(self.fig, self.update_frame, frames=frame_ids, 
                                      init_func=self.init_animation, blit=False, repeat=False)
        return ani

# Query for tracking data of the specific play
game_id, play_id = 2022102307, 641
# game_id, play_id = 2022102308, 185
# game_id, play_id = 2022102302, 970
# game_id, play_id = 2022102302, 2606
ball_snap_frameid = df_tracking.query('game_id == @game_id & play_id == @play_id & event == "ball_snap"')['frame_id'].iloc[0]
frames = range(ball_snap_frameid - 5, ball_snap_frameid + 1)
tracking_data = df_tracking.query('game_id == @game_id & play_id == @play_id & frame_id.isin(@frames)').copy() # & frame_id.isin(@frames)').copy()

play_cols = ['home_team_logo', 'away_team_logo', 'play_clock_at_snap', 'game_clock', 
             'absolute_yardline_number', 'yards_to_go', 'away_team_color', 
             'home_team_color', 'possession_team', 'defensive_team', 'down_and_dist', 
             'quarter_with_suffix', 'pre_snap_home_score', 'pre_snap_visitor_score',
             'possession_team_color', 'defensive_team_color', 'home_team_abbr', 'away_team_abbr',
             'home_team_wordmark', 'possession_team_color2', 'defensive_team_color2']
play_data = df_play.query('game_id == @game_id & play_id == @play_id')[play_cols].to_dict(orient='records')[0]

mpl.rcParams['animation.embed_limit'] = 50

# Instantiate and generate the animation
play_anim = NFLPlayAnimation(tracking_data, play_data, show_scoreboard=True, clock_rolling=True, player_display_type='jerseys', show_player_legend=True)
ani = play_anim.animate_play()

# Use tight layout to minimize padding, and ensure the plot extends to the figure's edges
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

# Set the figure background to light grey
play_anim.fig.patch.set_facecolor('lightgray')

plt.close(play_anim.fig)

HTML(ani.to_jshtml(fps=10))

In [20]:
to_rgba('#A71930')

(0.6549019607843137, 0.09803921568627451, 0.18823529411764706, 1.0)

In [33]:
df_tracking.head()

,game_id,play_id,nfl_id,week,display_name,frame_id,frame_type,time,jersey_number,club,play_direction,x,y,s,a,dis,o,dir,event,position
0,2022102400,66,37087.0,7,Robert Quinn,1,BEFORE_SNAP,2022-10-25 00:16:40.6,94.0,CHI,right,27.93,28.63,1.29,0.98,0.12,200.77,179.85,huddle_break_offense,OLB
1,2022102400,66,37087.0,7,Robert Quinn,2,BEFORE_SNAP,2022-10-25 00:16:40.7,94.0,CHI,right,27.80,28.63,1.38,0.76,0.13,197.13,179.67,NaN,OLB
2,2022102400,66,37087.0,7,Robert Quinn,3,BEFORE_SNAP,2022-10-25 00:16:40.8,94.0,CHI,right,27.66,28.63,1.45,0.57,0.14,195.71,178.97,NaN,OLB
3,2022102400,66,37087.0,7,Robert Quinn,4,BEFORE_SNAP,2022-10-25 00:16:40.9,94.0,CHI,right,27.52,28.63,1.46,0.27,0.14,195.71,179.12,NaN,OLB
4,2022102400,66,37087.0,7,Robert Quinn,5,BEFORE_SNAP,2022-10-25 00:16:41,94.0,CHI,right,27.37,28.64,1.46,0.13,0.14,197.27,178.39,NaN,OLB


In [ ]:
play_data['possession_team_color2']

str

Todo:
- player legend
- adjust score on touchdown
- create process to plot and save plays (increase icloud storage or on memory storage)


In [54]:
df_player.head()

,nfl_id,height,weight,birth_date,college_name,position,display_name
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan


In [38]:
df_play.query('game_id==@game_id & play_id==@play_id').play_description.values[0]

'(7:44) Br.Hall left end for 62 yards, TOUCHDOWN.'